# Exercise 1

Retrieve the <b>legal name</b>, the residence <b>country</b> and the <b>ultimate parent</b> (LEI and name) of all companies present in the provided datasets (*issuances.csv* and *holdings.csv*), and store them in a dataframe.

To structure the requests, please find "lei-records" endpoint details [GLEIF API](https://documenter.getpostman.com/view/7679680/SVYrrxuU?version=latest#5fc83577-f5ae-41fa-aa4e-2848554958b4).

---

In [7]:
import pandas as pd
import requests
session = requests.Session()

In [8]:
issuances_df = pd.read_csv('../data/issuances.csv', sep=';')
holdings_df = pd.read_csv('../data/holdings.csv', sep=";")
url = "https://api.gleif.org/api/v1/lei-records/"

In [9]:
children_companies = list(set(holdings_df["holder_lei"].values.tolist()))

In [10]:
len(children_companies)

402

Since the GET request has a rate limiting (see [here](https://www.postman.com/planetary-moon-167465/workspace/is/collection/1849746-f4ede923-1a55-4b4d-a86d-5bb588e50298)), we need to use some tool to avoid getting the loop broken by an error. In fact, API providers constrain developers from making too many API calls, this may lead to your application to be be banned from making further API calls if it abuses these limits. In this case I decided to use a function decorator preventing a function from being called more often than that allowed by the API provider: [ratelimit](https://pypi.org/project/ratelimit/). [1]()

In [11]:
from ratelimit import limits, sleep_and_retry

# 60 calls per 60 seconds
CALLS = 60
RATE_LIMIT = 60

@sleep_and_retry
@limits(calls=CALLS, period=RATE_LIMIT)
def check_limit():
    ''' 
    Check number of calls to API in given period
    '''
    return

In [12]:
result = []
for child in children_companies:
    check_limit()
    response_child = session.get(url + child , headers={"Accept": "application/vnd.api+json"})
    name = response_child.json()["data"]["attributes"]["entity"]["legalName"]["name"]
    country = response_child.json()["data"]["attributes"]["entity"]["legalAddress"]["country"]
    response_parent = session.get(url + child + "/" +  "ultimate-parent", headers={"Accept": "application/vnd.api+json"})
    ultimate_parent_lei = response_parent.json()["data"]["id"]
    ultimate_parent_name = response_parent.json()["data"]["attributes"]["entity"]["legalName"]["name"]
    result.append((child, name, country, ultimate_parent_lei, ultimate_parent_name))


In [13]:
result_df = pd.DataFrame(result, columns=["child_lei", "child_name", "child_country", "parent_lei", "parent_name"])

In [14]:
result_df

,child_lei,child_name,child_country,parent_lei,parent_name
0,549300HDCRTBPEURI316,"UNILEVER ČR, spol. s r.o.",CZ,549300MKFYEKVRWML317,UNILEVER PLC
1,549300EF9ZLCS32PSN92,DEAM CAPITAL PROTECT 2019,DE,7LTWFZYICNSX8D621K86,DEUTSCHE BANK AKTIENGESELLSCHAFT
2,529900UUPS52ZOCZKF69,Deutsche Capital Partners China Limited,KY,7LTWFZYICNSX8D621K86,DEUTSCHE BANK AKTIENGESELLSCHAFT
3,549300L70BS183Y6ML67,DWS Investment S.A.,LU,7LTWFZYICNSX8D621K86,DEUTSCHE BANK AKTIENGESELLSCHAFT
4,549300SE1UII3HYCP132,MIXHOLD B.V.,NL,549300MKFYEKVRWML317,UNILEVER PLC
...,...,...,...,...,...
397,549300G0GSODGZH94Z69,VISION DIRECT EUROPE LTD,GB,549300M3VH1A3ER1TB49,ESSILORLUXOTTICA
398,5299000ALLPGKHTTUV74,Deutsche Bank Capital Trust I,US,7LTWFZYICNSX8D621K86,DEUTSCHE BANK AKTIENGESELLSCHAFT
399,3358007MD8NFWEKRC164,RAYBAN SUN OPTICS INDIA PRIVATE LIMITED,IN,549300M3VH1A3ER1TB49,ESSILORLUXOTTICA
400,5493002J445FKF3T3Q63,DWS INVEST - MULTI ASSET MODERATE INCOME,LU,7LTWFZYICNSX8D621K86,DEUTSCHE BANK AKTIENGESELLSCHAFT


In [15]:
result_df.to_csv('result_exercise_1.csv', index=False)

References:
- [1]() ratelimit 2.2.1 [https://pypi.org/project/ratelimit/](https://pypi.org/project/ratelimit/)